In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from langchain_groq import ChatGroq

model = ChatGroq(model="openai/gpt-oss-120b", temperature=0)

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt="You are a travel agent. No follow up questions."
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Get me a direct flight from San Francisco to Tokyo on March 31st")]},
    config
    )

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Get me a direct flight from San Francisco to Tokyo on March 31st', additional_kwargs={}, response_metadata={}, id='e346cf08-9a5c-4fe4-a8e5-adc17ecdaaaf'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to get a direct flight from SFO to Tokyo on March 31. Use search-flight. Provide cabin class default economy, currency default EUR maybe. Need direct flight only. The API may have a parameter for max stopovers? Not in spec. But we can filter results after retrieving. We\'ll request and then filter for flights with no layovers (i.e., route length 1). The tool returns flights; we need to display table grouped by cheapest, shortest, rest. Also need to include only direct flights. We\'ll request with flyFrom "San Francisco" (SFO) and flyTo "Tokyo" (any airport). Use departureDate "31/03/2026". No return date. Cabin class M. We\'ll call function.', 'tool_calls': [{'id': 'fc_92f9861e-0fde-4a69-aee2-5db14593c6ea', 'functi

In [7]:
print(response["messages"][-1].content)

**Direct flights SFO → Tokyo (NRT) on 31 Mar 2026**

| Route | Departure → Arrival (local) & duration | Cabin | Price (EUR) | Book |
|-------|----------------------------------------|-------|-------------|------|
| **Cheapest** | SFO → NRT <br>31/03 16:45 → 01/04 20:00 (11 h 15 m) | Economy | **570** | [Book](https://on.kiwi.com/w8V2Yb) |
| **Shortest** | SFO → NRT <br>31/03 12:20 → 01/04 15:25 (11 h 5 m) | Economy | **1 062** | [Book](https://on.kiwi.com/rlzzn3) |
| **Other options** | SFO → NRT <br>31/03 11:40 → 01/04 15:00 (11 h 20 m) | Economy | 881 | [Book](https://on.kiwi.com/nteKHh) |
|  | SFO → NRT <br>31/03 12:20 → 01/04 15:25 (11 h 5 m) | Economy | 1 099 | [Book](https://on.kiwi.com/cmqagy) |

### Summary & Recommendation
- **Best price:** €570 – a comfortable 11 h 15 m flight, departing late afternoon and arriving the next day in the evening.  
- **Fastest direct:** €1 062 – only 5 minutes shorter (11 h 5 m) and leaves around midday.  
- If you prefer an earlier arrival in T